# Text Preprocessing:

Data cleaning is an important and intensive process in Data science which aids in data analysis and building machine learning models.

In [21]:
import pandas as pd
import unidecode 
import re
import time 
import stopwords 
# nltk.download('stopwords') 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk import word_tokenize
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

import langid
from langdetect import DetectorFactory
from langdetect import detect
from langdetect import detect_langs

import warnings
warnings.filterwarnings('ignore')

In [22]:
def get_data_from_csv(url):
    df = pd.read_csv(url)
    return df

# Now we start preparing the data:

- Lowercase the text
- Remove links
- Remove non english language
- remove punctuation, stop words

https://github.com/kk7nc/Text_Classification

# Stop words
Text and document classification over social media, such as Twitter, Facebook, and so on is usually affected by the noisy nature (abbreviations, irregular forms) of the text corpuses.


# Capitalization
Sentences can contain a mixture of uppercase and lower case letters. Multiple sentences make up a text document. To reduce the problem space, the most common approach is to reduce everything to lower case. This brings all words in a document in same space, but it often changes the meaning of some words, such as "US" to "us" where first one represents the United States of America and second one is a pronoun. To solve this, slang and abbreviation converters can be applied.

# Noise Removal
Another issue of text cleaning as a pre-processing step is noise removal. Text documents generally contains characters like punctuations or special characters and they are not necessary for text mining or classification purposes. Although punctuation is critical to understand the meaning of the sentence, but it can affect the classification algorithms negatively.

In [23]:
def cleaning_preprocessing_data_from_csv(data):  
    cleaned_data = []
    
    for text in data:

        # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
        formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

        # Removing all the occurrences of links that starts with https
        formatted_text = re.sub(r'http\S+', '', formatted_text)

        # Remove all the occurrences of text that ends with .com
        formatted_text = re.sub(r"\ [A-Za-z]*\.com", " ", formatted_text)

        # Remove all whitespaces
        pattern = re.compile(r'\s+') 
        formatted_text = re.sub(pattern, ' ', formatted_text)
        formatted_text = formatted_text.replace('?', ' ? ').replace(')', ') ')

        # Remove accented characters from text using unidecode.
        # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
        remove_character = unidecode.unidecode(formatted_text)

        # Convert text to lower case
        lower_text = remove_character.lower()

        # Pattern matching for all case alphabets
        Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)

        # Limiting all the  repeatation to two characters.
        formatted_text = Pattern_alpha.sub(r"\1\1", lower_text) 

        # Pattern matching for all the punctuations that can occur
        Pattern_Punct = re.compile(r'(\'[.,/#!"$<>@[]^&%^&*?;:{}=_`~()+-])\1{1,}')

        # Limiting punctuations in previously formatted string to only one.
        Combined_Formatted = Pattern_Punct.sub(r'\1', formatted_text)

        # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
        Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)

        # The formatted text after removing not necessary punctuations.
        Formatted_Text = re.sub(r"[^a-zA-Z]+", ' ', Final_Formatted) 
        
#         # Lemmatization is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item
#         wordnet = WordNetLemmatizer()
#         Formatted_Text =  " ".join([wordnet.lemmatize(word) for word in Formatted_Text])
#         print("Finish lemmatizer")

        # Text without stopwords
        remove_stop_words = repr(Formatted_Text)
        stoplist = stopwords.words('english') 

        # Append words to Medium.com
        stoplist.extend(['ago', 'followers', 'pinned', 'read', 'min', 'published', 'days', 'hours', 'the'])
            
        No_StopWords = [word for word in word_tokenize(remove_stop_words) if word.lower() not in stoplist ]

        # Convert list of tokens_without_stopwords to String type.
        words_string = No_StopWords[0]
        words_string = ' '.join(No_StopWords[1:]) 
        
        # Remove more stop words  
        final_text = remove_stopwords(words_string) 
        
        # Split the "'" from the edges
        cleaned_data.append(final_text[:len(final_text)-1])
        
    return cleaned_data

In [24]:
def remove_non_english_articles_and_remove_duplicated_rows(df):
    # dropping ALL duplicate values (keep only one)
    df.drop_duplicates('content', inplace = True)
    
#     print("DETECT TOPICS:")
#     for topic in df["topic"]:
#         DetectorFactory.seed = 0
#         if detect(topic) != "en":
#             print(topic)
#             print("Found different language: " + detect(topic))
#             df.drop(df.index[(df["topic"] == topic)], axis=0, inplace=True)
    
#     print("DETECT CONTENTS:")
#     for content in df["content"]:
#         DetectorFactory.seed = 0
#         if detect(content) != "en":
#             print("Found different language: " + detect(content))
#             df.drop(df.index[(df["content"] == content)], axis=0, inplace=True) #axis 0 for rows
    
#     print("DETECT CATEGORIES:")
#     for category in df["category"]:   
#         DetectorFactory.seed = 0
#         if detect(topic) != "en":
#             print(category)
#             print("Found different language: " + detect(topic))
#             df.drop(df.index[(df["category"] == category)], axis=0, inplace=True)
            

    return df

In [25]:
def drop_rows_with_short_content(df):
    # also drop nulls
    # split values by whitespace and drop data lt a word
    df = df[df["topic"].str.split().str.len() > 1]
    
    # split values by whitespace and drop data lt a word and bt 5
    df = df[df["category"].str.split().str.len() < 5]
    df = df[df["category"].str.split().str.len() > 0]

    # split values by whitespace and drop data lt 3 words 
    df = df[df["content"].str.split().str.len() > 3]
    
    return df

## Keep the 'cleaned' data frame

In [26]:
def insert_df_to_csv(topic, category, content, csv_file_name):
    new_df = pd.DataFrame({
#     "topic": topic,
    "category": category,
    "content": content
    })
    new_df.to_csv(csv_file_name)
    return new_df

In [27]:
df = get_data_from_csv("BBC-News-Train.csv")
print("Before cleaning data the shape of data frame is : " + str(df.shape))
df

Before cleaning data the shape of data frame is : (1490, 3)


,Unnamed: 0,content,category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [28]:
df = drop_rows_with_short_content(df)
print("After drop_rows_with_short_content the shape of data frame is : " + str(df.shape))

df = remove_non_english_articles_and_remove_duplicated_rows(df)
print("After remove_non_english_articles_and_remove_duplicated_rows the shape of data frame is : " + str(df.shape))

# def lemmatize_word(text):
#     wordnet = WordNetLemmatizer()
#     return " ".join([wordnet.lemmatize(word) for word in text])
# df['content'] = df['content'].apply(lemmatize_word)
# df
title_list = cleaning_preprocessing_data_from_csv(df["topic"])
print("Finish cleaning on topic")
content_list = cleaning_preprocessing_data_from_csv(df["content"])
print("Finish cleaning on content")

After drop_rows_with_short_content the shape of data frame is : (1490, 3)
After remove_non_english_articles_and_remove_duplicated_rows the shape of data frame is : (1440, 3)
Finish cleaning on content


In [30]:
def lower_category_and_union_categories_with_same_meaning():
    category_list = []
    for category in df["category"]:
        lower_category = category.lower()
        if lower_category == "sports":
            lower_category = "sport"
        if lower_category == "health-and-fitness":
            lower_category = "health-fitness"
        if lower_category == "arts-and-entertainment":
            lower_category = "arts-entertainment"
        if lower_category == "computers-and-technology" or lower_category == "computers-technology":
            lower_category = "computers"
        if lower_category == "travel-and-leisure" or lower_category == "travel-leisure" or lower_category == "travel-and-tourism-lifestyle-and-leisure":
            lower_category = "travel"
        if lower_category == "news-and-society" or lower_category == "news-society":
            lower_category = "society"
        if lower_category == "food-and-drink":
            lower_category = "food"
        category_list.append(lower_category)

    print(len(category_list))
    return category_list

In [32]:
category_list = lower_category_and_union_categories_with_same_meaning()
new_df = insert_df_to_csv(title_list, category_list, content_list, "cleaned-scraping-data.csv")
print("After cleaning data the shape of data frame is : " + str(new_df.shape))

new_df

1440
After cleaning data the shape of data frame is : (1440, 2)


,category,content
0,business,ex boss launches defence lawyers defending wor...
1,business,business confidence slides german business con...
2,business,poll indicates economic gloom citizens majorit...
3,tech,governs mobile choice faster better funkier ha...
4,business,bosses payout eighteen enron directors agreed ...
...,...,...
1435,entertainment,eviction big brother model caprice holby city ...
1436,entertainment,double act revamp chart dj duo jk joel taking ...
1437,business,dollar hits reuters revenues media group reute...
1438,tech,ipod family expands market apple expanded ipod...


In [33]:
print(new_df['category'].value_counts())

sport            342
business         335
politics         266
entertainment    263
tech             234
Name: category, dtype: int64
